In [1]:
import org.apache.hadoop.fs.{FileSystem, Path}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{StructType, StructField, StringType, ArrayType}

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,application_1701051468713_0009,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.hadoop.fs.{FileSystem, Path}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{StructType, StructField, StringType, ArrayType}


In [2]:
val schema1 = StructType(Array(
    StructField("anonymousId", StringType, true),
    StructField("context", StringType, true),
    StructField("messageId", StringType, true),
    StructField("properties", StringType, true),
    StructField("event", StringType, true),
    StructField("timestamp", StringType, true),
    StructField("type", StringType, true),
    StructField("userId", StringType, true)
))

val schema2 = StructType(Array(
    StructField("userAgent", StringType, true),
    StructField("ip", StringType, true),
    StructField("locale", StringType, true),
    StructField("page", StringType, true)
))

val schema3 = StructType(Array(
    StructField("url", StringType, true)
))

val schema4 = StructType(Array(
    StructField("path", StringType, true),
    StructField("page", StringType, true),
    StructField("button", StringType, true),
    StructField("action", StringType, true),
    StructField("properties", StringType, true)
))

val schema5 = StructType(Array(
    StructField("name", StringType, true),
    StructField("hospital", ArrayType(StringType), true)
))

val schema6 = StructType(Array(
    StructField("hospitalId", StringType, true),
    StructField("pageName", StringType, true),
    StructField("routeName", StringType, true),
    StructField("to", StringType, true)
))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schema1: org.apache.spark.sql.types.StructType = StructType(StructField(anonymousId,StringType,true),StructField(context,StringType,true),StructField(messageId,StringType,true),StructField(properties,StringType,true),StructField(event,StringType,true),StructField(timestamp,StringType,true),StructField(type,StringType,true),StructField(userId,StringType,true))
schema2: org.apache.spark.sql.types.StructType = StructType(StructField(userAgent,StringType,true),StructField(ip,StringType,true),StructField(locale,StringType,true),StructField(page,StringType,true))
schema3: org.apache.spark.sql.types.StructType = StructType(StructField(url,StringType,true))
schema4: org.apache.spark.sql.types.StructType = StructType(StructField(path,StringType,true),StructField(page,StringType,true),StructField(button,StringType,true),StructField(action,StringType,true),StructField(properties,StringType,true))
schema5: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,true),StructF

In [27]:
def src_to_segment_log(df: org.apache.spark.sql.DataFrame) : org.apache.spark.sql.DataFrame = {
    val dF = df.withColumn("schema1", from_json(col("value"), schema1)).
        select(col("schema1.*"), col("partition_0")).
        withColumn("schema2", from_json(col("context"), schema2)).
        select(col("anonymousId"), col("schema2.*"), col("context"), col("messageId"), col("properties"), col("event")
                , col("timestamp"), col("type"), col("userId"), col("partition_0")).
        withColumn("schema3", from_json(col("page"), schema3)).
        withColumn("context_useragent", col("userAgent")).
        withColumn("context_ip", col("ip")).
        withColumn("context_locale", col("locale")).
        withColumn("properties_", col("properties")).
        select(col("anonymousId"), col("context_useragent"), col("context_ip"), col("context_locale"), col("schema3.*"), col("context")
                , col("messageId"), col("properties_"), col("event"), col("timestamp"), col("type"), col("userId"), col("partition_0")).
        withColumn("schema4", from_json(col("properties_"), schema4)).
        withColumn("context_page_url", col("url")).
        select(col("anonymousId"), col("context_useragent"), col("context_ip"), col("context_locale"), col("context_page_url"), col("context")
                , col("messageId"), col("schema4.*"), col("properties_"), col("event"), col("timestamp"), col("type"), col("userId"), col("partition_0")).
        withColumn("schema5", from_json(col("button"), schema5)).
        withColumn("properties_path", col("path")).
        withColumn("properties_page", col("page")).
        select(col("anonymousId"), col("context_useragent"), col("context_ip"), col("context_locale"), col("context_page_url"), col("context")
                , col("messageId"), col("properties_path"), col("properties_page"), col("schema5.*"), col("action"), col("properties")
                , col("properties_"), col("event"), col("timestamp"), col("type"), col("userId"), col("partition_0")).
        withColumn("properties_button_hospital_num", col("hospital").getItem(0)).
        withColumn("properties_button_hospital_name", col("hospital").getItem(1)).
        withColumn("properties_button_name", col("name")).
        drop(col("button"), col("hospital"), col("name")).
        withColumn("schema5", from_json(col("action"), schema5)).
        select(col("anonymousId"), col("context_useragent"), col("context_ip"), col("context_locale"), col("context_page_url"), col("context")
                , col("messageId"), col("properties_path"), col("properties_page"), col("properties_button_name")
                , col("properties_button_hospital_num"), col("properties_button_hospital_name"), col("schema5.*"), col("properties")
                , col("properties_"), col("event"), col("timestamp"), col("type"), col("userId"), col("partition_0")).
        withColumn("properties_action_hospital_id", col("hospital").getItem(0)).
        withColumn("properties_action_hospital_name", col("hospital").getItem(1)).
        withColumn("properties_action_name", col("name")).
        drop(col("action"), col("hospital"), col("name")).
        withColumn("schema6", from_json(col("properties"), schema6)).
        select(col("anonymousId"), col("context_useragent"), col("context_ip"), col("context_locale"), col("context_page_url"), col("context")
                , col("messageId"), col("properties_path"), col("properties_page"), col("properties_button_name"), col("properties_action_name")
                , col("properties_action_hospital_id"), col("properties_action_hospital_name")
                , col("properties_button_hospital_num"), col("properties_button_hospital_name")
                , col("schema6.*"), col("properties_"), col("event"), col("timestamp"), col("type"), col("userId"), col("partition_0")).
        withColumn("properties_properties_hospitalid", col("hospitalId")).withColumn("properties_properties_pagename", col("pageName")).
        withColumn("properties_properties_routename", col("routeName")).withColumn("properties_properties_to", col("to")).
        withColumn("properties", col("properties_")).
        drop(col("hospitalId"), col("pageName"), col("routeName"), col("to"), col("properties_")).
        withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'")).
        withColumn("yw", concat(year(date_add(from_utc_timestamp(col("timestamp"), "Asia/Seoul"), 1)), lit("-")
                                 , lpad(weekofyear(date_add(from_utc_timestamp(col("timestamp"), "Asia/Seoul"), 1)), 2, "0")))
    
    dF.createOrReplaceTempView("dF")
    val df_segment_log = spark.sql(f"""
        SELECT anonymousId, context_useragent, context_ip, context_locale, context_page_url, context,
        messageId, properties_path, properties_page, properties_button_name, properties_action_name, properties_action_hospital_id,
        properties_action_hospital_name, properties_button_hospital_num, properties_button_hospital_name, properties_properties_hospitalid,
        properties_properties_pagename, properties_properties_routename, properties_properties_to, properties,
        event, from_utc_timestamp(timestamp, 'Asia/Seoul') AS timestamp,
        yw,
        date_add(date_trunc('week', date_add(from_utc_timestamp(timestamp, 'Asia/Seoul'), 1)), -1) AS ywd,
        date_format(from_utc_timestamp(timestamp, 'Asia/Seoul'), 'yyyy-MM') AS ym,
        date_format(from_utc_timestamp(timestamp, 'Asia/Seoul'), 'yyyy-MM-dd') AS ymd,
        type, userId, partition_0
        FROM dF
    """)
    
    return df_segment_log
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

src_to_segment_log: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [9]:
def segment_log_to_segment_log_session(df_segment_log: org.apache.spark.sql.DataFrame) : org.apache.spark.sql.DataFrame= {
    df_segment_log.createOrReplaceTempView("df_segment_log")
    
    val df_segment_log_session = spark.sql("""
        SELECT
            event.anonymousid || '-' || row_number() over(partition by event.anonymousid order by event.timestamp) as session_id,
            event.anonymousid,
            event.timestamp as session_start_at,
            lead(timestamp) over(partition by event.anonymousid order by event.timestamp) as next_session_start_at
        FROM (
            SELECT
                e.anonymousid,
                e.timestamp,
                DATEDIFF(minute, LAG(e.timestamp) OVER(PARTITION BY e.anonymousid ORDER BY e.timestamp), e.timestamp) AS inactivity_time
            FROM df_segment_log AS e) as event
        WHERE (event.inactivity_time > 30 OR event.inactivity_time is null)
    """)
    
    return df_segment_log_session
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

segment_log_to_segment_log_session: (df_segment_log: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [10]:
def get_s3_directories(s3_uri: String) : Array[String] = {
    val fs = FileSystem.get(new java.net.URI(s3_uri), spark.sparkContext.hadoopConfiguration)

    val directories = fs.listStatus(new Path(s3_uri)).filter(_.isDirectory).map(_.getPath.toString)
    
    return directories
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

get_s3_directories: (s3_uri: String)Array[String]


In [33]:
def save_to_redshift(df: org.apache.spark.sql.DataFrame, table_name: String) = {
    val str_jdbc_url="jdbc:redshift://fastcampus-workgroup.515958955371.ap-northeast-2.redshift-serverless.amazonaws.com:5439/dev?user=team04&password=0xHBH5VX"
    val str_s3_path="s3://emr-data-sync/team04/redshift_tempdir"
    
    df.write.mode("append").
    format("io.github.spark_redshift_community.spark.redshift").
    option("url", str_jdbc_url).option("dbtable", table_name).
    option("forward_spark_s3_credentials", "true").
    option("tempdir", str_s3_path).save()
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

save_to_redshift: (df: org.apache.spark.sql.DataFrame, table_name: String)Unit


In [34]:
val bucket = "emr-data-sync/"
val key = "segment-logs/61pktWj9yLj7tBBUBPUCCf"

val directories = get_s3_directories("s3a://" + bucket + key)

directories.foreach(
    directory => 
    {
        println(directory)
        val partition_0 = directory.split("/").last
        val df = spark.read.text(directory + "/*.gz").withColumn("partition_0", lit(partition_0))
        val df_segment_log = src_to_segment_log(df)
        val df_segment_log_session = segment_log_to_segment_log_session(df_segment_log)

        save_to_redshift(df_segment_log, "segment_log")
        save_to_redshift(df_segment_log_session, "segment_log_session")
    }
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucket: String = emr-data-sync/
key: String = segment-logs/61pktWj9yLj7tBBUBPUCCf
directories: Array[String] = Array(s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1684454400000, s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1684540800000, s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1684627200000, s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1684713600000, s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1684800000000, s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1684886400000, s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1684972800000, s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1685059200000, s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1685145600000, s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1685318400000, s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPU...
s3a://emr-data-sync/segment-logs/61pktWj9yLj7tBBUBPUCCf/1684454400000
s3a://emr-data-sync/segment-logs/61pktWj9yLj7tB

In [40]:
val jdbcDF_log = spark.read.format("io.github.spark_redshift_community.spark.redshift").
  option("url", str_jdbc_url).
  option("dbtable", "segment_log").
  option("tempdir", str_s3_path).
  option("forward_spark_s3_credentials", "true").
  load()


jdbcDF_log.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

jdbcDF_log: org.apache.spark.sql.DataFrame = [anonymousid: string, context_useragent: string ... 27 more fields]
+--------------------+--------------------+---------------+--------------+--------------------+--------------------+--------------------+---------------+--------------------+----------------------+----------------------+-----------------------------+-------------------------------+------------------------------+-------------------------------+--------------------------------+------------------------------+-------------------------------+------------------------+--------------------+--------------------+--------------------+-------+----------+-------+----------+-----+--------------------+-------------+
|         anonymousid|   context_useragent|     context_ip|context_locale|    context_page_url|             context|           messageid|properties_path|     properties_page|properties_button_name|properties_action_name|properties_action_hospital_id|properties_action_hospital_n

In [41]:
val jdbcDF_log_session = spark.read.format("io.github.spark_redshift_community.spark.redshift").
  option("url", str_jdbc_url).
  option("dbtable", "segment_log_session").
  option("tempdir", str_s3_path).
  option("forward_spark_s3_credentials", "true").
  load()


jdbcDF_log_session.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

jdbcDF_log_session: org.apache.spark.sql.DataFrame = [session_id: string, anonymousid: string ... 2 more fields]
+--------------------+--------------------+--------------------+---------------------+
|          session_id|         anonymousid|    session_start_at|next_session_start_at|
+--------------------+--------------------+--------------------+---------------------+
|6cf9264f-d613-463...|6cf9264f-d613-463...|2023-05-19 16:03:...|                 null|
|f84d1075-86dd-4f4...|f84d1075-86dd-4f4...|2023-05-19 16:04:...| 2023-05-19 17:26:...|
|f84d1075-86dd-4f4...|f84d1075-86dd-4f4...|2023-05-19 17:26:...|                 null|
|9c1fb78e-bba5-4de...|9c1fb78e-bba5-4de...|2023-05-19 19:31:...|                 null|
|a62b87e4-d13d-492...|a62b87e4-d13d-492...|2023-05-19 20:30:...|                 null|
|9347d745-6b03-498...|9347d745-6b03-498...|2023-05-19 15:42:...| 2023-05-19 16:53:...|
|9347d745-6b03-498...|9347d745-6b03-498...|2023-05-19 16:53:...|                 null|
|a9b3fc16-6b76-4f